In [1]:
import sys

In [ ]:
import findspark
findspark.init()
import pyspark
import random

In [2]:
from elasticsearch import Elasticsearch

In [17]:
es = Elasticsearch()

#### Run the follwing command in the terminal(to authenticate) to get the mail archive before running the next commands
gmvault sync you@gmail.com -d path/to/where/you/want/the/email/archived

In [15]:
!cat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.meta

{"thread_ids": 1349228336576455316, "msg_id": "1785262635.1286724410050.JavaMail.async@sl-prod-sso-eas5-1", "flags": ["\\Seen"], "gm_id": 1349228336576455316, "internal_date": 1286706411, "x_gmail_received": null, "labels": ["\\Inbox"], "subject": "Welcome to your Nokia account"}

In [ ]:
!gzcat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.eml.gz

### ===================================
We’ll walk all the files in the root of the Gmvault database using os.walk, find all files that end in .meta, load the JSON in those files, tweak the JSON just a bit (more on that in a second), and then shove the JSON into Elasticsearch.

In [3]:
import os, json, timeit
import email
import gzip
import io

In [4]:
#converting all characters to unicode, as elasticsearch handles only unicode
def toUnicode(char):
    return char.decode('latin-1', errors='replace')

In [22]:
#can just store the mapping keys instead of the whole document,and when searched, 
#may access the data using the mapping key. This may be done by storing the actual data somewhere else, 
#maybe hadoop or not

def parse_and_store(es, root, email_path):
    gm_id = os.path.split(email_path)[-1]
    
    with gzip.open(email_path +'.eml.gz', 'r') as fp:
        message = email.message_from_file(fp)
        #change all the content to unicode for convenience of elasticsearch
        meta = {toUnicode(key).lower(): toUnicode(value) 
                for key, value in message.items()}
                
        with open(email_path  + '.meta', 'r') as fp:
            meta.update(json.load(fp))
            
        #getting content of mail
        content = io.StringIO()
        #we'll extract only text from the mail content
        if message.is_multipart():
            for part in message.get_payload():
                if part.get_content_type() == 'text/plain':
                    content.write(toUnicode(part.get_payload()))
        else:
            content.write(toUnicode(message.get_payload()))
                
        #add additional attributes to meta[_'source'], 
        #all new additions are made implicitly inside the meta['_source']
        meta['account'] = 'rohit.mlhtra@gmail.com'
        meta['path'] = email_path
                
        #add complete data as body
        mail = meta.copy()
        mail['content'] = content.getvalue()
        
        es.index(index='mail', doc_type = 'mail', id = gm_id, body=mail)
        #es.index(index = 'mail_mes', doc_type = 'message', id = gm_id, body=body)

In [23]:
def index(root):
    es = Elasticsearch()
    root = os.path.abspath(root)
    for parent_folder, folders, files in os.walk(root):
        #f = FloatProgress(min = 0, max = len(files))
        #display(f)
        for filename in files:
            #f.value += 1
            if filename.endswith('.meta'):
                parse_and_store(es, root, os.path.join(parent_folder, filename.split('.')[0]))

In [26]:
from ipywidgets import FloatProgress
from IPython.display import display

folder_path = '/Users/GodSpeed/Documents/mailArchive/'
index(folder_path)

## Searching the metadata

##### Lazy search(non structured query)

In [18]:
es.search('mail', q='playstation')

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 131}

#### To do: 
1. sync offline mails regularly

#### Parsing emails text

In [ ]:
import email
import gzip

In [ ]:
!gzcat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.eml.gz

In [ ]:
with gzip.open('/Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.eml.gz', 'r') as fp:
    message = email.message_from_file(fp)
print "%r"%(message.items())